# Equal Weight S&P500

### Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import warnings

### Importing List of Stocks
Using s&p500_updated as it is the updated list of s&p500 companies. This avoids using the try and except block later on.

In [2]:
stocks = pd.read_csv("s&p500_updated.csv")

### Acquiring an API Token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

### Making an API Call

1) base url get it from IEX Cloud documentation <br>
2) we want the **"quote"** from the API because this gives us: <br>
    - market capitalisation <br>
    - price of each stock  <br>

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

### Parsing the data

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

### Adding Stocks Data to Pandas Data Frame

In [6]:
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'No. of Shares to buy']
dataframe = pd.DataFrame(columns=my_columns)

### To avoid the FutureWarning when appending onto the pd DataFrame

In [7]:
warnings.simplefilter(action='ignore', category=FutureWarning)

### Appending the AAPL data onto the dataframe (testing)

In [8]:
aapl_data = pd.Series(['AAPL', price, market_cap, 'N/A'], index=my_columns) #index=my_columns alligns the data to the columns
dataframe = dataframe.append(aapl_data, ignore_index=True) #can only append a series if ignore_index=True

### Looping through all the Tickers

Made use of try and except block to take into consideration the S&P500 list: <br>
    - The S&P500 list is not constant as companies can be added or removed. If the API call doesn't work that means the company is no longer part of the S&P500 and thus we price it at 0 and give it a market capitalisation of 0 so it's not taken into consideration.

In [9]:
dataframe = pd.DataFrame(columns=my_columns)
for symbol in stocks['Ticker']:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
    try:
        data = requests.get(api_url).json()
        price = data['latestPrice']
        market_cap = data['marketCap']
    except ValueError:
        price = 0
        market_cap = 0
    ticker_data = pd.Series([symbol, data['latestPrice'], data['marketCap'], 'N/A'], index=my_columns)
    dataframe = dataframe.append(ticker_data, ignore_index=True)

### Using Batch API Calls to improve performance
Splitting the 500 into 5 batches to make 5 API calls instead of 500

In [10]:
'''Function sourced from https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks'''
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
ticker_groups = list(chunks(stocks['Ticker'], 100))
batch_apis = []
for i in range(len(ticker_groups)):
    batch_apis.append(",".join(ticker_groups[i]))

final_dataframe = pd.DataFrame(columns=my_columns)

for batch_api in batch_apis:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={batch_api}&token={IEX_CLOUD_API_TOKEN}"
    #batch api url is different to normal api urls. See documentation on IEX Clous
    data = requests.get(batch_api_url).json()
    tickers = batch_api.split(',')
    for ticker in tickers:
        price = data[ticker]['quote']['latestPrice']
        market_cap = data[ticker]['quote']['marketCap']
        ticker_data = pd.Series([ticker, price, market_cap, 'N/A'], index=my_columns)
        final_dataframe = final_dataframe.append(ticker_data, ignore_index=True)

In [12]:
final_dataframe

,Ticker,Price,Market Capitalization,No. of Shares to buy
0,MMM,124.22,69094458997,N/A
1,AOS,56.54,8720784414,N/A
2,ABT,108.83,193098627206,N/A
3,ABBV,168.47,293887886278,N/A
4,ABMD,389.34,17697530893,N/A
...,...,...,...,...
497,YUM,130.24,36523249076,N/A
498,ZBRA,250.94,13133887145,N/A
499,ZBH,128.35,27507533740,N/A
500,ZION,47.75,7086301214,N/A


### Calculating the Number of Shares to buy

Implemented a reccursive function to allow the user to input how much money they want to invest. Then the following function will return how many of each shares should be bought to get an equal weighting as the S&P500

In [13]:
val = input("How much money would you like to invest? ($) ")
value = []
def money(val):
    try:
        num = float(val)
    except:
        num = "error"
        
    if type(num) == float:
        print("Thank you")
        value.append(num)
    else:
        val = input("Please input a number ")
        return money(val)

money(val)

How much money would you like to invest? ($) 100000
Thank you


In [14]:
invest = value[0]

def equal_weight_sp500(invest):
    '''Calculates how many shares to get of each stock'''
    ratio = invest / len(final_dataframe)
    for i in range(0, len(final_dataframe)):
        final_dataframe.loc[i, 'No. of Shares to buy'] = ratio // final_dataframe['Price'][i]
    return final_dataframe

In [15]:
equal_weight_sp500(invest)

,Ticker,Price,Market Capitalization,No. of Shares to buy
0,MMM,124.22,69094458997,1.0
1,AOS,56.54,8720784414,3.0
2,ABT,108.83,193098627206,1.0
3,ABBV,168.47,293887886278,1.0
4,ABMD,389.34,17697530893,0.0
...,...,...,...,...
497,YUM,130.24,36523249076,1.0
498,ZBRA,250.94,13133887145,0.0
499,ZBH,128.35,27507533740,1.0
500,ZION,47.75,7086301214,4.0


### Using the xlsxwriter to format the output into an excel file

In [16]:
writer = pd.ExcelWriter("Recommended_Trades.xlsx", engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name="Recommended Trades", index=False)

In [17]:
background_color = '#FFFFFF'
font_color = '#000000'

string_format = writer.book.add_format({'font_color': font_color, 'bg_color': background_color})

dollar_format = writer.book.add_format({'num_format':'$0.00', 'font_color': font_color, 
                                        'bg_color': background_color})

integer_format = writer.book.add_format({'num_format':'0','font_color': font_color,
                                         'bg_color': background_color, 'border':1})


In [18]:
column_formats = {'A': ['Ticker', string_format],'B': ['Price', dollar_format],
                  'C': ['Market Capitalization', dollar_format], 'D': ['Number of Shares to Buy', integer_format]}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [19]:
writer.save()